NOTE: This notebook uses a JSON dictionary created by the `ndlar_flow/scripts/low_level_data_ana/data_sim_comparisons/Event_Level_Hit_Metrics_Dict_Builder.py` script. As this notebook comprises of preliminary/diagnostic studies, the code may not be fully optimized. The code will be optimized in terms of runtime and organization as time permits. Please contact Elise Hinkle (ehinkle@uchicago.edu) with any questions, comments, or concerns. Also, all JSON dictionary file locations are likely outdated and should be changed based on where the user has chosen to store the relevant dictionary(ies).

In [1]:
# Import relevant packages/libraries
import matplotlib as mlp
import matplotlib.pyplot as plt
import numpy as np
import h5py
import argparse
from datetime import datetime
import glob
import json
import cmasher as cmr
import math
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec
import sys
import os
import sys
sys.path.append('/global/cfs/cdirs/dune/users/ehinkle/nd_prototypes_ana/2x2_sim/run-ndlar-flow/ndlar_flow/event_display/LAr_evd/')
from lar2x2_evd import *
from collections import Counter

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Hello


In [2]:
# Open dictionary giving hit-level event summary information
# Note that this dictionary only includes summary information for BEAM EVENTS
events_data_file = '/global/cfs/cdirs/dune/users/ehinkle/nd_prototypes_ana/2x2_sim/run-ndlar-flow/ndlar_flow/event_display/LAr_evd/event_hits_dict_V5.json'
events_data_open = open(events_data_file)
events_data_dict = json.load(events_data_open)


# Make two new dictionaries for BEAM events with unphysical and/or unexpected max hit drift times
# Only take events with negative max hit drift time (i.e. make new dictionary with only these events)
negative_max_hit_drift_time_dict = {k: v for k, v in events_data_dict.items() if v['max_hit_drift_time'] < 0}
print('Number of events with negative max hit drift time: ', len(negative_max_hit_drift_time_dict))
# Only take events with HIGH max hit drift time (i.e. make new dictionary with only these events)
high_max_hit_drift_time_dict = {k: v for k, v in events_data_dict.items() if v['max_hit_drift_time'] > 2100}
print('Number of events with high max hit drift time: ', len(high_max_hit_drift_time_dict))

Number of events with negative max hit drift time:  18
Number of events with high max hit drift time:  2


In [6]:
# Quick check to see how many events have max hit drift time > 2100 and how many hits they have
for v in events_data_dict.values():
    if v['max_hit_drift_time'] > 2100:
        print("Event ID:", v['event_id'])
        print("Number of Hits:", v['nhits'])
        print('---------------------------------')


Event ID: 3261
Number of Hits: 3
---------------------------------
Event ID: 1638
Number of Hits: 306
---------------------------------


In [17]:
# Make a PDF showing event displays of events with negative max hit drift times. The PDF includes one "normal" event display 
# and one event display with negative drift time hits highlighted with lime * markers.
output_pdf_name = 'negative_max_hit_drift_time_events.pdf'
with PdfPages(output_pdf_name, keep_empty=False) as output:
    counter = 0
    for keys, values in negative_max_hit_drift_time_dict.items():
        counter += 1
        #if counter>2: break
        if counter%5 == 0: print('--------------- LOOKING AT EVENT NUMBER: ', counter, '---------------')
        file = values['filepath'].split('/')[-1]
        directory = values['filepath'].split('packet')[0]
        ev_id = values['event_id']

        evd = LArEventDisplay(filedir=directory, filename=file, nhits_min=values['nhits']-1, ntrigs=0, show_light=False, show_colorbars=True, beam_only=True)
        hits_ini, cmap_ini, charge_norm_ini = evd.display_event(ev_id)
        output.savefig(bbox_inches='tight')
        #plt.close()
        hits, cmap, charge_norm = evd.display_event(ev_id)
        f = h5py.File(directory+file,'r')
        events = f['charge/events/data']
        event_mask = events['id'] == ev_id
        event = events[event_mask]
        hits_ref = f['charge/events/ref/charge/calib_prompt_hits/ref']
        hits_region = f['charge/events/ref/charge/calib_prompt_hits/ref_region']
        #packets_full = f['charge/packets/data']
        #packets_ref = f['charge/calib_prompt_hits/ref/charge/packets/ref']
        #packets_hit_ref = f['charge/calib_prompt_hits/ref/charge/packets/ref']
        hit_ref = hits_ref[hits_region[ev_id,'start']:hits_region[ev_id,'stop']]
        hit_ref = np.sort(hit_ref[hit_ref[:,0] == ev_id, 1])
        #packet_ref = packets_ref[hit_ref]
        #packet_ref = np.sort(packet_ref[:, 1])
        #packets = packets_full[packet_ref]
        #adc_sat_idx = np.where(packets['packet_type']==0) and np.where(packets['dataword']==255)
        #adc_sat_hits_ref = np.sort(packets_hit_ref[:,0][adc_sat_idx])
        #sat_hits = hits[adc_sat_hits_ref]
        negative_time_hits = hits[hits['t_drift']<0]
        evd.ax_bdv.scatter(negative_time_hits['z'], negative_time_hits['x'], negative_time_hits['y'], c='lime', s=25, alpha=0.50, marker='*', label=str(len(negative_time_hits))+' Negative Drift Time Hits')
        evd.ax_bd.scatter(negative_time_hits['z'], negative_time_hits['x'], c='lime', s=25, alpha=0.50, marker='*')
        evd.ax_bv.scatter(negative_time_hits['z'], negative_time_hits['y'], c='lime', s=25, alpha=0.50, marker='*')
        evd.ax_dv.scatter(negative_time_hits['x'], negative_time_hits['y'], c='lime', s=25, alpha=0.50, marker='*')
        evd.ax_bdv.legend()
        output.savefig(bbox_inches='tight')
        plt.close()

Number of available events: 456
For fast-forwarding purposes, here is every 10th event number in your sample: [117, 235, 348, 510, 640, 754, 872, 977, 1135, 1231, 1319, 1393, 1493, 1571, 1657, 1752, 1828, 1933, 2027, 2106, 2188, 2270, 2402, 2477, 2554, 2630, 2733, 2809, 2880, 2976, 3060, 3163, 3235, 3304, 3385, 3489, 3566, 3659, 3746, 3832, 3943, 4016, 4113, 4191, 4282]
Number of external triggers in this event: 1
Event 1925 is a beam trigger event
Number of available events: 456
For fast-forwarding purposes, here is every 10th event number in your sample: [117, 235, 348, 510, 640, 754, 872, 977, 1135, 1231, 1319, 1393, 1493, 1571, 1657, 1752, 1828, 1933, 2027, 2106, 2188, 2270, 2402, 2477, 2554, 2630, 2733, 2809, 2880, 2976, 3060, 3163, 3235, 3304, 3385, 3489, 3566, 3659, 3746, 3832, 3943, 4016, 4113, 4191, 4282]
Number of external triggers in this event: 1
Event 1925 is a beam trigger event
Number of available events: 245
For fast-forwarding purposes, here is every 10th event number 

In [18]:
# Make a PDF showing event displays of events with high max hit drift times (over expected drift time of 201 us). The PDF includes one "normal" event display 
# and one event display with high drift time (>210 us) hits highlighted with aquamarine * markers.
output_pdf_name = 'high_max_hit_drift_time_events.pdf'
with PdfPages(output_pdf_name, keep_empty=False) as output:
    counter = 0
    for keys, values in high_max_hit_drift_time_dict.items():
        counter += 1
        #if counter>2: break
        if counter%5 == 0: print('--------------- LOOKING AT EVENT NUMBER: ', counter, '---------------')
        file = values['filepath'].split('/')[-1]
        directory = values['filepath'].split('packet')[0]
        ev_id = values['event_id']

        evd = LArEventDisplay(filedir=directory, filename=file, nhits_min=values['nhits']-1, ntrigs=0, show_light=False, show_colorbars=True, beam_only=True)
        hits_ini, cmap_ini, charge_norm_ini = evd.display_event(ev_id)
        output.savefig(bbox_inches='tight')
        #plt.close()
        hits, cmap, charge_norm = evd.display_event(ev_id)
        f = h5py.File(directory+file,'r')
        events = f['charge/events/data']
        event_mask = events['id'] == ev_id
        event = events[event_mask]
        hits_ref = f['charge/events/ref/charge/calib_prompt_hits/ref']
        hits_region = f['charge/events/ref/charge/calib_prompt_hits/ref_region']
        #packets_full = f['charge/packets/data']
        #packets_ref = f['charge/calib_prompt_hits/ref/charge/packets/ref']
        #packets_hit_ref = f['charge/calib_prompt_hits/ref/charge/packets/ref']
        hit_ref = hits_ref[hits_region[ev_id,'start']:hits_region[ev_id,'stop']]
        hit_ref = np.sort(hit_ref[hit_ref[:,0] == ev_id, 1])
        #packet_ref = packets_ref[hit_ref]
        #packet_ref = np.sort(packet_ref[:, 1])
        #packets = packets_full[packet_ref]
        #adc_sat_idx = np.where(packets['packet_type']==0) and np.where(packets['dataword']==255)
        #adc_sat_hits_ref = np.sort(packets_hit_ref[:,0][adc_sat_idx])
        #sat_hits = hits[adc_sat_hits_ref]
        high_drift_time_hits = hits[hits['t_drift']>210]
        evd.ax_bdv.scatter(high_drift_time_hits['z'], high_drift_time_hits['x'], high_drift_time_hits['y'], c='aquamarine', s=25, alpha=0.50, marker='*', label=str(len(high_drift_time_hits))+' >210 us Drift Time Hits')
        evd.ax_bd.scatter(high_drift_time_hits['z'], high_drift_time_hits['x'], c='aquamarine', s=25, alpha=0.50, marker='*')
        evd.ax_bv.scatter(high_drift_time_hits['z'], high_drift_time_hits['y'], c='aquamarine', s=25, alpha=0.50, marker='*')
        evd.ax_dv.scatter(high_drift_time_hits['x'], high_drift_time_hits['y'], c='aquamarine', s=25, alpha=0.50, marker='*')
        evd.ax_bdv.legend()
        output.savefig(bbox_inches='tight')
        plt.close()

Number of available events: 436
For fast-forwarding purposes, here is every 10th event number in your sample: [117, 205, 300, 374, 463, 579, 664, 746, 835, 947, 1011, 1091, 1172, 1265, 1384, 1462, 1538, 1616, 1727, 1808, 1890, 1973, 2106, 2192, 2257, 2353, 2442, 2555, 2634, 2708, 2791, 2867, 2997, 3071, 3164, 3250, 3364, 3439, 3523, 3629, 3751, 3848, 3932]
Number of external triggers in this event: 1
Event 3261 is a beam trigger event
Number of available events: 436
For fast-forwarding purposes, here is every 10th event number in your sample: [117, 205, 300, 374, 463, 579, 664, 746, 835, 947, 1011, 1091, 1172, 1265, 1384, 1462, 1538, 1616, 1727, 1808, 1890, 1973, 2106, 2192, 2257, 2353, 2442, 2555, 2634, 2708, 2791, 2867, 2997, 3071, 3164, 3250, 3364, 3439, 3523, 3629, 3751, 3848, 3932]
Number of external triggers in this event: 1
Event 3261 is a beam trigger event
Number of available events: 256
For fast-forwarding purposes, here is every 10th event number in your sample: [147, 303, 4